In [1]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def read_json(json_file) -> dict:
    with open(json_file, "r") as f:
        dictionary = json.load(f)
    return dictionary

def read_csv(csv_file):
    dataframe = pd.read_csv(csv_file)
    return dataframe

# Función para calcular el F1-score
def calculate_f1_score(precision, recall):
    # Evitar división por cero
    return np.where((precision + recall) > 0, 2 * (precision * recall) / (precision + recall), 0)

def combinar_datagramas(training_dfs):
    # Crear una lista para almacenar los DataFrames con MultiIndex
    dfs = []
    for training_id, df in training_dfs.items():
        # Añadir el identificador del entrenamiento como columna temporal
        df = df.copy()
        df['tune_id'] = training_id
        # Establecer un MultiIndex con 'tune_id' y 'epoch'
        df = df.set_index(['tune_id', df.index])
        dfs.append(df)

    # Combinar todos los DataFrames en uno solo
    combined_df = pd.concat(dfs)

    # Renombrar el índice de épocas
    combined_df.index.names = ['tune_id', 'epoch']
    
    # Añadir F1_score
    combined_df['metrics/F1_score(B)'] = calculate_f1_score(combined_df['metrics/precision(B)'], combined_df['metrics/recall(B)'])
    combined_df['metrics/F1_score(M)'] = calculate_f1_score(combined_df['metrics/precision(M)'], combined_df['metrics/recall(M)'])

    # Reordenar las columnas para colocar F1_score entre recall y mAP50
    # Obtener la lista de columnas
    columns = combined_df.columns.tolist()

    # Encontrar las posiciones de las columnas relevantes
    pos_recall_B = columns.index('metrics/recall(B)')
    pos_recall_M = columns.index('metrics/recall(M)')

    # Extraer las columnas F1_score recién creadas
    f1_b = combined_df.pop('metrics/F1_score(B)')
    f1_m = combined_df.pop('metrics/F1_score(M)')

    # Insertar las columnas en las posiciones deseadas
    combined_df.insert(pos_recall_B + 1, 'metrics/F1_score(B)', f1_b)
    combined_df.insert(pos_recall_M + 1, 'metrics/F1_score(M)', f1_m)

    return combined_df

def return_best_case(tunes_result, metrica):
    best_cases = pd.DataFrame()

    for tune, results in tunes_result.items():
        max_index = results[metrica].idxmax()
        # Obtener los detalles del caso con el mayor valor
        max_row = results.loc[max_index, ["metrics/precision(M)", "metrics/recall(M)", "metrics/F1_score(M)", "metrics/mAP50(M)", "metrics/mAP50-95(M)", "epoch", "trial_id", "model"]]
        best_cases[tune] = max_row
    
    best_cases = best_cases[sorted(best_cases.columns)]
    return best_cases

In [2]:
# tune: model
casos_tune = {"tune": "yolov8l", "tune2": "yolov8x", "tune3": "yolov9e",
              "tune4": "yolo11l", "tune5": "yolo11x", "tune6": "yolov8l"}

tunes_path = "runs/detect"
tunes_result = {}

# Recorrer los distintos archivos tune
for tune in os.listdir(tunes_path):
    tune_path = os.path.join(tunes_path, tune)
    # Entrar en la carpeta tune con fecha
    for date in os.listdir(tune_path):
        tune_full_path = os.path.join(tune_path, date)
        # Crear un diccionario donde guardar los dataframes de cada entrenamiento
        training_dfs = {}

        # Recorrer todos los tune dentro de la carpeta
        for train in os.listdir(tune_full_path):
            train_path = os.path.join(tune_full_path, train)

            # Ignorar los archivos y solo revisar las carpetas
            if os.path.isdir(train_path):
                tune_id = "_".join(train.split("_")[2:4])
                params_path = os.path.join(train_path, "params.json")
                result_path = os.path.join(train_path, "progress.csv")
                params = read_json(params_path)
                dataframe = read_csv(result_path)
                training_dfs[tune_id] = dataframe

        # Combinar los datagramas del diccionario en uno solo
        combined_df = combinar_datagramas(training_dfs)
        if tune in casos_tune.keys():
            combined_df["model"] = casos_tune[tune]
        else:
            combined_df["model"] = "Unknown"

    # Añadir el datagrama combinado al diccionario de tunes.
    tunes_result[tune] = combined_df

In [3]:
tunes_result["tune"].columns

Index(['metrics/precision(B)', 'metrics/recall(B)', 'metrics/F1_score(B)',
       'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'metrics/precision(M)',
       'metrics/F1_score(M)', 'metrics/recall(M)', 'metrics/mAP50(M)',
       'metrics/mAP50-95(M)', 'val/box_loss', 'val/seg_loss', 'val/cls_loss',
       'val/dfl_loss', 'epoch', 'timestamp', 'checkpoint_dir_name', 'done',
       'training_iteration', 'trial_id', 'date', 'time_this_iter_s',
       'time_total_s', 'pid', 'hostname', 'node_ip', 'time_since_restore',
       'iterations_since_restore', 'model'],
      dtype='object')

In [4]:
best_map5095 = return_best_case(tunes_result, "metrics/mAP50-95(M)")
best_f1score = return_best_case(tunes_result, "metrics/F1_score(M)")

In [5]:
best_f1score

,tune,tune2,tune3,tune4,tune5,tune6
metrics/precision(M),0.99997,0.99686,0.98747,0.99989,1.0,1.0
metrics/recall(M),0.97468,1.0,1.0,0.97468,0.98451,0.9745
metrics/F1_score(M),0.987163,0.998428,0.993696,0.987124,0.992195,0.987085
metrics/mAP50(M),0.98616,0.995,0.99462,0.98752,0.99452,0.9888
metrics/mAP50-95(M),0.81582,0.80951,0.8447,0.84026,0.82736,0.82396
epoch,59,23,49,66,39,77
trial_id,80d97_00012,43e31_00028,5ef43_00043,2e04e_00031,c1a22_00013,e52e8_00044
model,yolov8l,yolov8x,yolov9e,yolo11l,yolo11x,yolov8l


In [6]:
best_map5095

,tune,tune2,tune3,tune4,tune5,tune6
metrics/precision(M),0.99908,1.0,0.99829,0.99873,0.99918,0.99968
metrics/recall(M),0.97468,0.97165,0.94937,0.96203,0.97468,0.97468
metrics/F1_score(M),0.986729,0.985621,0.973216,0.980037,0.986778,0.987022
metrics/mAP50(M),0.98517,0.98242,0.98139,0.98815,0.99049,0.98853
metrics/mAP50-95(M),0.85227,0.85667,0.86713,0.85897,0.86155,0.85723
epoch,68,66,80,66,64,68
trial_id,80d97_00014,43e31_00007,5ef43_00022,2e04e_00002,c1a22_00000,e52e8_00034
model,yolov8l,yolov8x,yolov9e,yolo11l,yolo11x,yolov8l
